In [1]:
from Bio import SeqIO
import numpy as np
temp_sequences = []
for record in SeqIO.parse("/home/samridhi/boatgang/sequences.fa", "fasta"):
    temp_sequences.append(str(record.seq))

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/home/samridhi/boatgang/labels.txt", sep="\t")

In [4]:
print temp_sequences[0]
print type(temp_sequences[0])

GGGCGATATTCACTGCTGGGACACATACTTGAAGTCAATCTCTGACTGAACAGATGGTCGGGAACTCCTTCACTCATACGATTTTAAAATCTAAGCAATCCTGTAGTTTGCGTACCTTCTCAAACCCGGAAGTGTTGACCGGTTCGTTGACGATAGAAATAAAAGCCTTGAATCTTTTTTTACATTAATTCAACCTAGTC
<type 'str'>


In [10]:
temp_target = df['task1']
temp_sequence = pd.DataFrame(temp_sequences)
temp_sequence = temp_sequence.join(temp_target)
temp_sequence.to_csv("mycsv.csv")

In [6]:
print temp_sequences[0]

GGGCGATATTCACTGCTGGGACACATACTTGAAGTCAATCTCTGACTGAACAGATGGTCGGGAACTCCTTCACTCATACGATTTTAAAATCTAAGCAATCCTGTAGTTTGCGTACCTTCTCAAACCCGGAAGTGTTGACCGGTTCGTTGACGATAGAAATAAAAGCCTTGAATCTTTTTTTACATTAATTCAACCTAGTC


In [7]:
dict_i = {"A" : 0, "C": 1, "G": 2, "T" : 3}

In [8]:
def one_hot_encode(s):
    i=0
    ohe = np.zeros((len(s), 4))
    for k in s:
        ohe[i,dict_i[k]] = 1
        i+=1
    return ohe

In [9]:
ohe_sequences = np.array([one_hot_encode(s) for s in temp_sequences])
ohe_sequences.shape

(60000, 200, 4)

In [22]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(ohe_sequences, temp_target, test_size = 0.25, random_state = 42)

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout,Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [36]:
model = Sequential()
model.add(Conv1D(filters=40, kernel_size=12, activation='relu', input_shape=(200,4)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 189, 40)           1960      
Total params: 1,960
Trainable params: 1,960
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.add(MaxPooling1D(pool_size=189, strides=1))

In [38]:
model.add(Dense(200,  activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())

In [39]:
model.add(Dense(1, activation = "sigmoid"))

In [40]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 189, 40)           1960      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 40)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1, 200)            8200      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 200)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 201       
Total params: 10,361
Trainable params: 10,361
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss='binary_crossentropy', optimizer='Adam',metrics=['accuracy'])

In [42]:
model.fit(train_x, train_y, epochs=10, verbose=1)

Epoch 1/10
45000/45000 [==============================] - 5s 118us/step - loss: 0.1140 - acc: 0.9599
Epoch 2/10
45000/45000 [==============================] - 5s 106us/step - loss: 0.0260 - acc: 0.9954
Epoch 3/10
45000/45000 [==============================] - 5s 109us/step - loss: 0.0237 - acc: 0.9958
Epoch 4/10
45000/45000 [==============================] - 5s 109us/step - loss: 0.0223 - acc: 0.9962
Epoch 5/10
45000/45000 [==============================] - 5s 115us/step - loss: 0.0215 - acc: 0.9961
Epoch 6/10
45000/45000 [==============================] - 5s 114us/step - loss: 0.0202 - acc: 0.9961
Epoch 7/10
45000/45000 [==============================] - 5s 113us/step - loss: 0.0196 - acc: 0.9962
Epoch 8/10
45000/45000 [==============================] - 5s 107us/step - loss: 0.0186 - acc: 0.9961
Epoch 9/10
45000/45000 [==============================] - 5s 108us/step - loss: 0.0179 - acc: 0.9961
Epoch 10/10
45000/45000 [==============================] - 5s 107us/step - loss: 0.0157 - a

In [43]:
scores = model.evaluate(test_x, test_y)
print ("Test loss ", scores[0])
print ("Test acc ", scores[1])

15000/15000 [==============================] - 1s 48us/step
('Test loss ', 0.027899882172172267)
('Test acc ', 0.995)
